In [6]:
 
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure
import py3Dmol
import os
from matplotlib.colors import LinearSegmentedColormap
from skimage.transform import resize
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio.PDB import PDBParser, PDBIO, Select
from PIL import Image
import biotite.structure.io.pdb as pdb
from collections import defaultdict
import glob
import random
from Bio.PDB import PDBParser, Vector
from Bio.PDB.vectors import rotaxis
from math import pi
from biotite.structure import AtomArray, find_aromatic_rings
from biotite.structure import BondList
import requests
import os
import time
import json
from tqdm import tqdm
from Bio.PDB import   PPBuilder,Chain, Model
from Bio.SeqUtils import seq1
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
import biotite.structure as struc
pdb_path=r'D:\PythonProj\Auto-EC\pdb_files\1FT5.pdb'

In [7]:
# Define common ions, solvents, and crystallization aids to remove
# This list might need refinement based on specific needs and whether metal ions are structural/functional
UNWANTED_HETATMS = {
    # Solvents"HOH","CA","SO4","IOD","NA","CL","GOL","PO4"
    "HOH", "WAT", "H2O", "DOD",
    # Common Ions (Consider if specific ions like ZN, MG, CA are part of the active site)
    "NA", "K", "CA", "MG", "ZN", "CU", "FE", "MN", "CO", "NI", "NA+", "K+", "CA+", "MG+", "ZN+", "CL-", "BR-",
    "CL", "BR", "IOD", "F",
    "SO4", "PO4", "NO3", "CO3", "ACY", "ACT", "EDO", "NH4",
    # Crystallization Aids / Buffers
    "GOL", "PEG", "PGE", "PG4", "P6G", "OLC", "BME", "MPD", "MES", "CAC",
    "CIT", "TAR", "TLA", "TRS", "EDTA", "IMD", "HEPES", "POU", "IPA", "PGA",
    # Add more as needed based on common contaminants or non-ligand molecules
}
# Van der Waals radii for common elements (in Angstroms)
vdwRadii = {
    'H': 1.2,
    'C': 1.7,
    'N': 1.55,
    'O': 1.52,
    'S': 1.8,
    'P': 1.8,
    'F': 1.47,
    'Cl': 1.75,
    'Br': 1.85,
    'I': 1.98,
    'Fe': 2.0,
    'Zn': 1.39,
    'Cu': 1.4,
    'Ni': 1.63
}


def load_pdb_and_extract_conect(pdb_file):
    """
    Load a PDB file and extract CONECT records
    
    Parameters:
    -----------
    pdb_file : str
        Path to the PDB file
        
    Returns:
    --------
    tuple
        (structure, conect_dict) where structure is a biotite structure object
        and conect_dict is a dictionary mapping atom serial numbers to their connected atoms
    """
    # Load the PDB file
    with open(pdb_file, 'r') as f:
        pdb_content = f.readlines()
    
    # Extract CONECT records
    conect_dict = defaultdict(list)
    for line in pdb_content:
        if line.startswith("CONECT"):
            # CONECT records format: "CONECT" followed by atom serial numbers
            # First number is the central atom, the rest are connected atoms
            fields = line.split()
            if len(fields) > 1:
                central_atom = int(fields[1])
                for i in range(2, len(fields)):
                    connected_atom = int(fields[i])
                    conect_dict[central_atom].append(connected_atom)
                    # Add reverse connection for undirected graph
                    conect_dict[connected_atom].append(central_atom)
    
    # Load structure using biotite
    structure = pdb.PDBFile.read(pdb_file)
    
    return structure, conect_dict

def get_ligand_atoms(structure, ligand_residue_id):
    """
    Extract atoms belonging to a specific ligand
    
    Parameters:
    -----------
    structure : biotite.structure.AtomArray
        Structure from PDB file
    ligand_residue_id : str
        Three-letter code of the ligand (e.g., 'HEM' for heme)
        
    Returns:
    --------
    dict
        Dictionary mapping atom indices to serial numbers
    """
    atom_indices = {}
    
    for i, residue in enumerate(structure.res_name):
        if residue == ligand_residue_id:
            atom_indices[structure.atom_id[i]] = i
    
    return atom_indices

def build_ligand_graph(structure, conect_dict, ligand_atoms):
    """
    Build a graph representation of the ligand using CONECT records
    
    Parameters:
    -----------
    structure : biotite.structure.AtomArray
        Structure from PDB file
    conect_dict : dict
        Dictionary mapping atom serial numbers to connected atoms
    ligand_atoms : dict
        Dictionary mapping atom serial numbers to atom indices
        
    Returns:
    --------
    networkx.Graph
        Graph representation of the ligand
    """
    G = nx.Graph()
    
    # Add nodes
    for atom_id in ligand_atoms:
        atom_idx = ligand_atoms[atom_id]
        atom_name = structure.atom_name[atom_idx]
        element = structure.element[atom_idx] if hasattr(structure, 'element') else atom_name[0]
        G.add_node(atom_id, name=atom_name, element=element)
    
    # Add edges from CONECT records
    for atom_id in ligand_atoms:
        if atom_id in conect_dict:
            for connected_atom in conect_dict[atom_id]:
                if connected_atom in ligand_atoms:
                    G.add_edge(atom_id, connected_atom)
    
    return G

def find_aromatic_ringsNX(G, min_ring_size=5, max_ring_size=7):
    """
    Find potential aromatic rings in the molecular graph
    
    Parameters:
    -----------
    G : networkx.Graph
        Molecular graph
    min_ring_size : int
        Minimum size of aromatic rings to consider
    max_ring_size : int
        Maximum size of aromatic rings to consider
        
    Returns:
    --------
    list
        List of lists containing atom IDs that form potential aromatic rings
    """
    # Find all cycles in the graph
    cycles = nx.cycle_basis(G)
    
    # Filter cycles by size
    potential_aromatic_rings = [cycle for cycle in cycles 
                               if min_ring_size <= len(cycle) <= max_ring_size]
    
    # Further filtering based on aromaticity criteria
    aromatic_rings = []
    for ring in potential_aromatic_rings:
        # Check if ring contains correct elements for aromaticity
        # For simplicity, we'll consider rings with C and N as potentially aromatic
        elements = [G.nodes[atom_id]['element'] for atom_id in ring]
        
        # Check if elements are mostly C, N, O, S (common in aromatic systems)
        aromatic_elements = [e for e in elements if e in ['C', 'N', 'O', 'S']]
        
        # If at least 75% of atoms are potential participants in aromaticity
        if len(aromatic_elements) / len(ring) >= 0.75:
            aromatic_rings.append(ring)
    
    return aromatic_rings

def is_heme(G, ring):
    """
    Check if a ring is part of a heme structure
    
    Parameters:
    -----------
    G : networkx.Graph
        Molecular graph
    ring : list
        List of atom IDs forming a ring
        
    Returns:
    --------
    bool
        True if the ring is likely part of a heme structure
    """
    # Heme typically has:
    # - Four pyrrole rings (five-membered rings with 4 C and 1 N)
    # - Iron (Fe) coordinated at the center
    
    # Check if the ring is a pyrrole (5-membered with 1 N and 4 C)
    if len(ring) != 5:
        return False
    
    elements = [G.nodes[atom_id]['element'] for atom_id in ring]
    if elements.count('N') != 1 or elements.count('C') != 4:
        return False
    
    # Check for iron nearby
    iron_nearby = False
    ring_neighbors = set()
    for atom_id in ring:
        ring_neighbors.update(G.neighbors(atom_id))
    
    ring_neighbors -= set(ring)  # Remove ring atoms
    
    for neighbor in ring_neighbors:
        if G.nodes[neighbor]['element'] == 'FE':
            iron_nearby = True
            break
    
    return iron_nearby

def analyze_pdb_for_aromatics(pdb_file, ligand_id):
    """
    Main function to analyze a PDB file for aromatic rings in a ligand
    
    Parameters:
    -----------
    pdb_file : str
        Path to the PDB file
    ligand_id : str
        Three-letter code of the ligand
        
    Returns:
    --------
    tuple
        (structure, aromatic_rings, heme_rings)
    """
    # Load PDB and extract CONECT records
    structure, conect_dict = load_pdb_and_extract_conect(pdb_file)
    
    # Get ligand atoms
    ligand_atoms = get_ligand_atoms(structure, ligand_id)
    
    # Build graph representation
    G = build_ligand_graph(structure, conect_dict, ligand_atoms)
    
    # Find aromatic rings
    aromatic_rings = find_aromatic_rings(G)
    
    # Identify heme rings
    heme_rings = [ring for ring in aromatic_rings if is_heme(G, ring)]
    
    # Remove heme rings from general aromatic rings
    aromatic_rings = [ring for ring in aromatic_rings if ring not in heme_rings]
    
    return structure, G, aromatic_rings, heme_rings



def CreateStack(model, ligand_info,boxSize_nm =2, gridSize =128 ):
    # Collect all backbone atoms
    backbone_atoms = []
    minX,minY,minZ = np.inf,np.inf,np.inf
    maxX,maxY,maxZ = -np.inf,-np.inf,-np.inf
    for chain in model:
        for residue in chain:
            if not residue.id[0] == " ":  # Skip heteroatoms
                continue
            resProps = residue_properties(residue)
            for atom in residue:
                if atom.id in ["N", "CA", "C", "O"]:  # Backbone atoms
                    atom.properties = resProps
                    backbone_atoms.append(atom)
                    # atom.coord = np.array(atom.coord)
                    # # Update min/max coordinates for bounding box
                    # minX = min(minX, atom.coord[0] - vdwRadii.get(atom.element, 1.2))
                    # minY = min(minY, atom.coord[1] - vdwRadii.get(atom.element, 1.2))
                    # minZ = min(minZ, atom.coord[2] - vdwRadii.get(atom.element, 1.2))
                    # maxX = max(maxX, atom.coord[0] + vdwRadii.get(atom.element, 1.2))
                    # maxY = max(maxY, atom.coord[1] + vdwRadii.get(atom.element, 1.2))
                    # maxZ = max(maxZ, atom.coord[2] + vdwRadii.get(atom.element, 1.2))
    
    boxSize_Ang=boxSize_nm*10
    maxX = ligand_info['center'][0] + boxSize_Ang/2
    maxY = ligand_info['center'][1] + boxSize_Ang/2
    maxZ = ligand_info['center'][2] + boxSize_Ang/2
    minX = ligand_info['center'][0] - boxSize_Ang/2
    minY = ligand_info['center'][1] - boxSize_Ang/2
    minZ = ligand_info['center'][2] - boxSize_Ang/2
                        
    grid3D_red = np.zeros((gridSize, gridSize, gridSize), dtype=np.float32)   
    grid3D_green = np.zeros((gridSize, gridSize, gridSize), dtype=np.float32)
    grid3D_blue = np.zeros((gridSize, gridSize, gridSize), dtype=np.float32)
    mask = np.zeros((gridSize, gridSize, gridSize), dtype=np.float32)
    
    voxelSizeX = (maxX - minX) / gridSize      
    voxelSizeY = (maxY - minY) / gridSize
    voxelSizeZ = (maxZ - minZ) / gridSize

    voxelSize = max(voxelSizeX, voxelSizeY, voxelSizeZ)

    for atomIndex  in range(len( backbone_atoms)):
        atom = backbone_atoms[atomIndex]
        x, y, z = atom.coord - np.array([minX, minY, minZ])
        
        if x < 0 or y < 0 or z < 0:
            continue
        if x>boxSize_Ang or y>boxSize_Ang or z>boxSize_Ang:
            continue
        x = int(x / voxelSize)
        y = int(y / voxelSize)
        z = int(z / voxelSize)
       
        radius = int( vdwRadii.get(atom.element, 1.2)/voxelSize)
        rsquared = radius * radius
        for dx in range(-radius, radius + 1):
            for dy in range(-radius, radius + 1):
                for dz in range(-radius, radius + 1):
                    if dx**2 + dy**2 + dz**2 <= rsquared:
                        xx=x + dx
                        yy=y + dy
                        zz=z + dz
                        if 0 <= xx < gridSize and 0 <= yy < gridSize and 0 <= zz < gridSize:
                            grid3D_red[xx, yy,zz] = x/ gridSize   
                            grid3D_green[xx, yy,zz] = atom.properties["polarity"]
                            grid3D_blue[xx, yy,zz] = atom.properties["size"]
                            pass
                            
    print('ligand drawing')
    for atom_prop in ligand_info['atom_properties']:
        #element, charge, polarity = atom_prop['element'], atom_prop['charge'], atom_prop['polarity']
        x, y, z = np.array(atom_prop['coords']) - np.array([minX, minY, minZ])
        
        if x < 0 or y < 0 or z < 0:
            continue
        if x > boxSize_Ang or y > boxSize_Ang or z > boxSize_Ang:
            continue
        x = int(x  / voxelSize)
        y = int(y  / voxelSize)
        z = int(z  / voxelSize)
       
        radius = int(vdwRadii.get(atom_prop['element'], 1.2) / voxelSize)
        rsquared = radius * radius
        for dx in range(-radius, radius + 1):
            for dy in range(-radius, radius + 1):
                for dz in range(-radius, radius + 1):
                    if dx**2 + dy**2 + dz**2 <= rsquared:
                        xx = x + dx
                        yy = y + dy
                        zz = z + dz
                        if 0 <= xx < gridSize and 0 <= yy < gridSize and 0 <= zz < gridSize:
                            grid3D_red[xx, yy, zz] = x/ gridSize   
                            grid3D_green[xx, yy, zz] = atom_prop['polarity']
                            grid3D_blue[xx, yy, zz] = atom_prop['aromatic']
                            mask[xx, yy, zz] = 1.0
    return grid3D_red, grid3D_green, grid3D_blue, mask, voxelSize, minX, minY, minZ       

def detect_rings(residue):
    """
    Detect rings in a residue using Biotite.
    
    Args:
        residue: Bio.PDB Residue object
    
    Returns:
        dict: A dictionary with atom IDs as keys and ring information as values
    """
    # Convert Bio.PDB residue to Biotite AtomArray
    atom_array = AtomArray(len(residue))
    atom_array.coord = np.array([atom.coord for atom in residue.get_atoms()])
    atom_array.element = np.array([atom.element.strip()[0] for atom in residue.get_atoms()])
    
    # Ensure the AtomArray has a BondList
    
    bond_list = BondList(atom_array.array_length())
    atom_list = list(residue.get_atoms())
    for i, atom1 in enumerate(atom_list):
        for j, atom2 in enumerate(atom_list):
            if i != j:
                distance = np.linalg.norm(atom1.coord - atom2.coord)
                # Use a simple heuristic to determine if atoms are bonded
                if distance < (vdwRadii.get(atom1.element.strip()[0], 1.2) + vdwRadii.get(atom2.element.strip()[0], 1.2)) * 0.6:
                    bond_list.add_bond(i, j)
    atom_array.bonds = bond_list
    
    
    # Detect aromatic rings
    aromatic_atoms = find_aromatic_rings(atom_array)
    
    ring_status = {}
    for atom in residue.get_atoms():
        atom_id = atom.get_id()
        ring_status[atom_id] = {
            "in_ring": atom_id in aromatic_atoms,
            "aromatic": atom_id in aromatic_atoms
        }
    # Handle special cases for hemes
    if residue.resname in ["HEM", "HEME"]:
        # Define the ring atoms in heme based on their names
        heme_ring_atoms = {"NA", "NB", "NC", "ND", "C1A", "C2A", "C3A", "C4A", 
                    "C1B", "C2B", "C3B", "C4B", "C1C", "C2C", "C3C", "C4C", 
                    "C1D", "C2D", "C3D", "C4D"}
        for atom in residue.get_atoms():
            atom_name = atom.get_name()
            ring_status[atom.get_id()] = {
                "in_ring": atom_name in heme_ring_atoms,
                "aromatic": atom_name in heme_ring_atoms  # Assume aromaticity for these atoms
            }
    return ring_status

def GetLigandInfo(ligand_info):
 

    residue = ligand_info['residue'] #bio.Pdb residue object
    atom_properties =[]
     
    
    # Analyze the residue structure
    ring_status = detect_rings(residue)
    
    
    for atom in residue.get_atoms(): #atom is bio.pdb atom not rdkit atom
        properties = {}
        element=atom.element 
        properties["element"] =  atom.element
        properties['coords'] = atom.coord
        
        # Simple heuristic for charge
        if element in ["O", "N", "F"]:
            properties["charge"] = -0.3  # Partial negative
        elif element in ["H"]:
            properties["charge"] = 0.1   # Partial positive (crude)
        else:
            properties["charge"] = 0.0

        # Estimate polarity based on electronegativity (simplified)
        if element in ["O", "N", "F"]:
            properties["polarity"] = 0.7  # Polar
        elif element in ["C", "H"]:
            properties["polarity"] = 0.3  # Non-polar
        else:
            properties["polarity"] = 0.5  # Intermediate

        properties["aromatic"] = 0.0
        # Check if the atom is in a ring and possibly aromatic
        if ring_status[atom.get_id()]['in_ring']:
            properties["aromatic"] = 1.0 if ring_status[atom.get_id()]['aromatic'] else 0.5
        else:
            properties["aromatic"] = 0.0
         
        # Metal binding potential 
        # Estimate how likely this atom is to interact with metal-binding residues
        element = atom.element
        #all metal ions are considered to have a high binding potential
        if element in ["FE", "Fe", "ZN", "Zn", "CU", "Cu", "NI", "Ni"]:
            properties["metal_binding"] = 1.0  # Iron can coordinate with His, Cys, etc.
        elif element in ["S"]:
            properties["metal_binding"] = 0.5  # Sulfur has strong metal coordination
        else:
            properties["metal_binding"] = 0.0  # Most other atoms have low metal binding potential
            
        #print(properties)
        atom_properties.append(properties)
    ligand_info['atom_properties'] = atom_properties
    return ligand_info

def residue_properties(residue):
    res_name = residue.resname
    properties = {}

    # Partial charge (estimated relative strength)
    if res_name in ["ARG", "LYS"]:
        properties["charge"] = 0.9  # High positive charge
    elif res_name == "HIS":
        properties["charge"] = 0.5  # Moderate positive charge
    elif res_name in ["ASP", "GLU"]:
        properties["charge"] = -0.9 # High negative charge
    else:
        properties["charge"] = 0.0  # Neutral

    # Aromaticity (boolean, but could be weighted by ring count)
    if res_name == "TRP":
        properties["aromatic"] = 1.0
    elif res_name== "PHE":
        properties["aromatic"] = 0.8 # Less aromaticity
    elif res_name == "TYR":
        properties["aromatic"] = 0.7
    elif res_name == "HIS":
        properties["aromatic"] = 0.5 # Less aromaticity
    else:
        properties["aromatic"] = 0.0

    # Size (approximate by residue number, could be improved with volume calc)
    num_atoms = len(residue)
    properties["size"] = num_atoms/15  # large

    # Polarity (estimated relative strength)
    if res_name in ["SER", "THR", "ASN", "GLN", "HIS", "CYS", "TYR"]:
        properties["polarity"] = 0.7  # Polar
    elif res_name in ["GLY", "ALA", "VAL", "LEU", "ILE", "PRO"]:
        properties["polarity"] = 0.3  # Non-polar
    else:
        properties["polarity"] = 0.5 #intermediate

    # Metal binding potential (heme binding, iron coordination)
    if res_name in ["HIS"]:
        properties["metal_binding"] = 0.9  # High potential for metal binding
    elif res_name in ["CYS", "MET"]:
        properties["metal_binding"] = 0.7
    elif res_name in ["ASP", "GLU"]:
        properties["metal_binding"] = 0.5  # Moderate to high potential
    elif res_name in ["THR", "SER", "TYR"]:
        properties["metal_binding"] = 0.2  # Moderate potential
    else:
        properties["metal_binding"] = 0 # Low potential for metal binding

    return properties

def get_ligands_and_interactions(structure,pdb_path):
    """
    Identify all ligands in PDB file.
    
    Args:
        pdb_path: Path to PDB file
        
    Returns:
        structure: Parsed PDB structure
        ligands_info: List of dictionaries with ligand information
    """
    model = structure[0]
    
    ligands_info = []
    for chain in model:
        for residue in chain:
            if residue.id[0] != ' ' :
                if len(residue)>5:
                    atomCoords = np.array([atom.coord for atom in residue])
                    #get the width, height, and depth of the ligand
                    min_coords = np.min(atomCoords, axis=0)
                    max_coords = np.max(atomCoords, axis=0)
                    width = max_coords[0] - min_coords[0]
                    height = max_coords[1] - min_coords[1]
                    depth = max_coords[2] - min_coords[2]
                    center = np.mean(atomCoords, axis=0)
                    # Get ligand info
                    ligand_info = {
                        'chain': chain.id,
                        'id': residue.id,
                        'name': residue.resname,
                        'residue': residue,
                        'atoms': len(residue),
                        'atoms_list': [atom.name for atom in residue],
                        'coordinates': atomCoords,
                        'width': width,
                        'height': height,
                        'depth': depth,
                        'center': center    ,
                    }
                
                    ligands_info.append(ligand_info)
    
    return  ligands_info

def loadPDF(pdb_path):    
    # Load the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_path)
    model = structure[0]
        
    # Generate random rotation angles in radians
    phi = random.uniform(0, 2*pi)  # Rotation around z-axis
    theta = random.uniform(0, 2*pi)  # Rotation around y-axis

    #Create rotation matrices
    rotation_z = rotaxis(phi, Vector(0, 0, 1))  # Rotation around z-axis
    rotation_y = rotaxis(theta, Vector(0, 1, 0))  # Rotation around y-axis
    translation = np.array((0.0, 0.0, 0.0), 'f')
    # Apply rotations to all atoms in the structure
    for atom in structure.get_atoms():
        # Apply first rotation around z-axis
        atom.transform(rotation_z,translation)
        # Then apply rotation around y-axis
        atom.transform(rotation_y,translation)
        
    ligands_info = get_ligands_and_interactions(structure,pdb_path)
    for info in ligands_info:
        GetLigandInfo(info)
        print(f"Name: {info['name']},  Width: {info['width']:.2f}, Height: {info['height']:.2f}, Depth: {info['depth']:.2f}")
    return structure, model, ligands_info
    
#display the loaded molecule  from the model object 
def displayModel(model):
    # Create a py3Dmol view
    view = py3Dmol.view(width=200, height=200)
    
    # Create a string representation of the PDB data
    pdb_string = ""
    for chain in model:
        for residue in chain:
            for atom in residue:
                # Format the atom line according to PDB standard
                line = f"ATOM  {atom.serial_number:5d} {atom.name:^4} {residue.resname:<3} {chain.id:1}{residue.id[1]:4d}    "
                line += f"{atom.coord[0]:8.3f}{atom.coord[1]:8.3f}{atom.coord[2]:8.3f}"
                line += f"  1.00  0.00          {atom.element:>2}\n"
                pdb_string += line
    
    # Add the PDB data to the view
    view.addModel(pdb_string, "pdb")
    
    # Style the protein
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.setStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon', 'radius': 0.3}})
    
    # Set view options
    view.zoomTo()
     
    
    # Show the view
    return view.show()


def create_depth_visualization(grid3D_red, grid3D_green, grid3D_blue, mask, num_sections=5):
    """
    Create a visualization of the 3D protein structure by slicing along the x-axis
    and applying dithering for transparency between sections.
    
    Parameters:
    -----------
    grid3D_red, grid3D_green, grid3D_blue : 3D numpy arrays
        The grid data representing different properties of the protein structure
    num_sections : int
        Number of depth sections to divide the x-axis into
    
    Returns:
    --------
    final_image : 2D numpy array
        The combined visualization of the protein structure
    """
    gridSize = grid3D_red.shape[0]
    section_size = gridSize // num_sections
    
    #fill with random ints up to num_sections
    dither = np.random.randint(1, num_sections, size=(gridSize, gridSize))
    # Create deterministic dithering patterns for each section
    dither_patterns = []
    for i in range(num_sections):
        expanded_pattern = (dither==i)
        dither_patterns.append(expanded_pattern)
    
    # Process each section from back to front
    final_rgb =np.zeros((gridSize, gridSize), dtype=np.float32)
    
    sections = [] 
    sumRedSections = [] 
    for section_idx in range(num_sections):
        start_idx = section_idx * section_size
        end_idx = min((section_idx + 1) * section_size, gridSize)
        
        # Extract meaningful data from this section by taking maximum values
        red_projection = np.max(  grid3D_red[start_idx:end_idx, :, :], axis=0) 
        green_projection = np.mean(grid3D_green[start_idx:end_idx, :, :], axis=0)
        blue_projection = np.mean(grid3D_blue[start_idx:end_idx, :, :], axis=0)
        mask_projection = np.max(mask[start_idx:end_idx, :, :], axis=0)
        final_rgb += red_projection
        sumRedSections.append(final_rgb+0)
        sections.append((red_projection, green_projection, blue_projection, mask_projection))
        
    final_rgb = np.zeros((gridSize, gridSize,3), dtype=np.float32)
    mask_ligand = np.zeros((gridSize, gridSize), dtype=np.float32)
    
    red_projection, green_projection, blue_projection, mask_projection = sections[0]
    final_rgb+=np.stack([red_projection, green_projection, blue_projection], axis=2)
    for section_idx in range(1,num_sections):
        red_projection, green_projection, blue_projection, mask_projection = sections[section_idx]
        dither=dither_patterns[section_idx]
        
        underSections= sumRedSections[section_idx-1] 
        hasInformationMask = (underSections==0)
        ditherOpen = dither+hasInformationMask
        
        # Use the dither pattern to selectively set pixels rather than adding values
        idx = (ditherOpen != 0)
        red = final_rgb[:,:,0]
        red[idx] =        red_projection[idx] 
        final_rgb[:,:,0] = red
        green = final_rgb[:,:,1]
        green[idx] =      green_projection[idx]
        final_rgb[:,:,1] = green
        blue = final_rgb[:,:,2]
        blue[idx] =       blue_projection[idx]
        final_rgb[:,:,2] = blue
        
        #green_projection[idx] = 0
        #blue_projection[idx] = 0
        #mask_projection[idx] = 0
         
        #final_rgb+=np.stack([red_projection, green_projection, blue_projection], axis=2)

        # # Similarly set the mask values only where dither is non-zero
        # mask_ligand += mask_projection
    return final_rgb,mask_ligand

def search_pdb_with_ligands(max_results=1000):
    """
    Search the RCSB PDB database for structures that contain ligands
    
    Parameters:
    -----------
    max_results : int
        Maximum number of results to return
        
    Returns:
    --------
    list
        List of PDB IDs that contain ligands
    """
    # Define the GraphQL query to search for structures with ligands
    # Excluding common ions and waters
    query = """
    query {
      search(
        type: CHEMISTRY_QUERY
        request: {
          queryType: FORMULA_QUERY
          formula: "C?"
          formulaType: FORMULA
          matchMode: STRICT
        }
        return_type: ENTRY
      ) {
        total_count
        entries {
          rcsb_id
        }
      }
    }
    """
    
    url = "https://data.rcsb.org/graphql"
    response = requests.post(url, json={'query': query})
    
    if response.status_code != 200:
        print(f"Error with GraphQL request: {response.status_code}")
        return []
    
    result = response.json()
    total_count = result['data']['search']['total_count']
    entries = result['data']['search']['entries']
    
    print(f"Found {total_count} structures with ligands")
    pdb_ids = [entry['rcsb_id'] for entry in entries[:max_results]]
    
    return pdb_ids

def get_ligand_info(pdb_id):
    """
    Get information about ligands in a PDB structure
    
    Parameters:
    -----------
    pdb_id : str
        PDB ID to query
        
    Returns:
    --------
    dict
        Dictionary with ligand information
    """
    url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error getting data for {pdb_id}: {response.status_code}")
        return None
    #print(json.dumps(response.json(), indent=4))    
    data = response.json()
    
    # Extract ligand information
    ligands = {}
    
    if 'rcsb_entry_info' in data and 'nonpolymer_entities' in data['rcsb_entry_info']:
        nonpolymer_count = data['rcsb_entry_info']['nonpolymer_entities']
        if nonpolymer_count > 0:
            # Get detailed entity information
            for entity_id in range(1, nonpolymer_count + 1):
                entity_url = f"https://data.rcsb.org/rest/v1/core/nonpolymer_entity/{pdb_id}/{entity_id}"
                entity_response = requests.get(entity_url)
                
                if entity_response.status_code == 200:
                    entity_data = entity_response.json()
                    if 'rcsb_nonpolymer_entity' in entity_data:
                        entity_info = entity_data['rcsb_nonpolymer_entity']
                        # Filter out common ions and water
                        if 'formula_weight' in entity_info and entity_info['formula_weight'] > 100:
                            ligand_id = entity_info.get('pdbx_entity_id', f"UNK{entity_id}")
                            ligand_name = entity_data.get('pdbx_entity_nonpoly', {}).get('comp_id', 'UNK')
                            formula = entity_info.get('formula', 'Unknown')
                            weight = entity_info.get('formula_weight', 0)
                            
                            ligands[ligand_id] = {
                                'name': ligand_name,
                                'formula': formula,
                                'weight': weight
                            }
    
    return {'pdb_id': pdb_id, 'ligands': ligands}

def download_pdb_file(pdb_id, output_dir=r'D:\PythonProj\pdbFiles\downloaded'):
    """
    Download a PDB file
    
    Parameters:
    -----------
    pdb_id : str
        PDB ID to download
    output_dir : str
        Directory to save the file
        
    Returns:
    --------
    str
        Path to the downloaded file
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_path = os.path.join(output_dir, f"{pdb_id}.pdb")
    
    # Check if file already exists
    if os.path.exists(output_path):
        print(f"File {output_path} already exists, skipping download")
        return output_path
    
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error downloading {pdb_id}: {response.status_code}")
        return None
    
    with open(output_path, 'wb') as f:
        f.write(response.content)
    
    return output_path
 
 




In [8]:
def get_identical_chains(model):
    # Dictionary to store sequences for each chain
    chain_sequences = {}
    
    # Get sequence for each chain using PPBuilder
    ppb = PPBuilder()
    for chain in model:
        # Get the sequence for this chain
        chain_seq = ""
        for pp in ppb.build_peptides(chain):
            chain_seq += str(pp.get_sequence())
        
        # Store the sequence if it's not empty
        if chain_seq:
            chain_sequences[chain.id] = chain_seq
    
    # Group chains by identical sequences
    identical_chains = {}
    processed_chains = set()
    
    for chain_id, seq in chain_sequences.items():
        if chain_id not in processed_chains:
            # Find all chains with identical sequence
            matching_chains=[]
            for c,s in chain_sequences.items():
                if c!=chain_id and abs( len(s)-len(seq))<4:
                    if s == seq:
                        matching_chains.append(c)
                    if len(s)>100:
                        sQ = seq[5:-5]
                        ss = s[5:-5]
                        if sQ in s:
                            matching_chains.append(c)
                            matching_chains.append(chain_id)
                        elif ss in seq:
                            matching_chains.append(c)
                            matching_chains.append(chain_id)
                    else:
                        if len(s)>len(seq):
                            if s[:len(seq)] == seq or s[-len(seq):] == seq:
                                matching_chains.append(c)
                                matching_chains.append(chain_id)
                        else:
                            if seq[:len(s)] == s or seq[-len(s):] == s:
                                matching_chains.append(c) 
                                matching_chains.append(chain_id)

                #if c != chain_id and s == seq:
                #    matching_chains.add(c)
            #matching_chains = [c for c, s in chain_sequences.items() if s == seq]
            if len(matching_chains) > 1:
                identical_chains[tuple(sorted(list(set(matching_chains))))] = seq
            processed_chains.update(matching_chains)
    

    return identical_chains




In [13]:
import biotite.structure.io.pdb as bio_pdb
def CensorProtein(model,identical_chains,    ligand_chain,  boxDistance_nm=2):
    #calculate radius of sphere around boxdistance cube
    cornerDist= np.sqrt( 3*(boxDistance_nm*10)**2)
    #find all residues with  
    # Add each ligand residue to the protein model
    
    ligandCenter = np.zeros(3)
    ligandAtomCount = 0
    for residue in ligand_chain:
        
        # Skip unwanted HETATM residues
        if residue.resname in UNWANTED_HETATMS:
            continue
        for atom in residue:
            ligandCenter += np.array(atom.coord)
            ligandAtomCount += 1
        
    if ligandAtomCount == 0:
        return None
    ligandCenter /= ligandAtomCount
    inDistanceResidues = []
    for chain in model:
        
        for residue in chain:
            if not residue.id[0] == " ":
                continue
            # Skip unwanted HETATM residues
            if residue.resname in UNWANTED_HETATMS:
                continue
            for atom in residue:
                # Calculate the distance from the ligand center to the atom
                distance = np.linalg.norm(np.array(atom.coord) - ligandCenter)
                if distance < cornerDist:
                    inDistanceResidues.append([residue,distance])
                    break
                
    # If there are identical chains, keep only residues from the closest chain(s)
    if identical_chains:
        # Map: chain_id -> count of close residues
        chain_contact_counts = {}
        chain_contact_norms = {}
        for chain_ids in identical_chains.keys():
            for cid in chain_ids:
                chain_contact_counts[cid] = 0
                chain_contact_norms[cid] = 0
        for pair in inDistanceResidues:
            residue, distance = pair
            cid = residue.get_parent().id
            if cid in chain_contact_counts:
                chain_contact_counts[cid] += (1/(.01+distance))
                chain_contact_norms[cid] += 1
                
        for chain_ids in chain_contact_counts.keys():
            norm = chain_contact_norms[chain_ids]
            if norm ==0:
                norm =1 
            chain_contact_counts[chain_ids] /= norm
            
        # For each identical group, keep only the chain with the most contacts
        keep_chains = set()
        for chain_ids in identical_chains.keys():
            max_contacts = -1
            best_chain = None
            for cid in chain_ids:
                if chain_contact_counts[cid] > max_contacts:
                    max_contacts = chain_contact_counts[cid]
                    best_chain = cid
            if best_chain is not None:
                keep_chains.add(best_chain)
        # Filter inDistanceResidues to keep only those from the selected chains
        inDistanceResidues = [res for res in inDistanceResidues if res[0].get_parent().id not in chain_contact_counts or res[0].get_parent().id in keep_chains]
    #return a new model with ligandchain and all residues within the distance of the ligand center
    new_model = Model.Model('zoom')
    proteinChain = Chain.Chain('A')
    id =0
    for residue in inDistanceResidues:
        newResidue = residue[0].copy()
        newResidue.id = (' ', id, ' ')
        #translate the atoms by ligandCenter
        for atom in newResidue:
            atom.coord -= ligandCenter
        proteinChain.add(newResidue)
        id += 1
       
        
    new_model.add(proteinChain)
    
    ligand_chain = ligand_chain.copy()
    for residue in ligand_chain:
        #translate the atoms by ligandCenter
        for atom in residue:
            atom.coord -= ligandCenter
    
    new_model.add(ligand_chain)
    return new_model
        

def load_MOAD_pdb(pdbID):
    
    #file format == 4eql_1_protein.pdb  (_1_ may be a more than 1 
    pdbPath = os.path.join(pdbFolder, f"{pdbID}_1_protein.pdb")
    ligandPath = os.path.join(ligandFolder, f"{pdbID}_1_superlig_")
    if not os.path.exists(pdbPath):
        pdbPath =  os.path.join(pdbFolder, f"{pdbID}_2_protein.pdb")
        ligandPath = os.path.join(ligandFolder, f"{pdbID}_2_superlig_")
        #2zox_1_superlig_2
    ligandFiles = []
    for i in range(4):
        if os.path.exists(f"{ligandPath}{i}.pdb"):
            ligandFiles.append(os.path.join(ligandFolder, f"{pdbID}_1_superlig_{i}.pdb"))
        
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdbPath)
    model = structure[0]
     
    identicalChains = get_identical_chains(model)
    cc=0
    for ligandFile in ligandFiles:    
        # print(ligandFile)
        # Load ligand with RDKit to get fingerprint
        # mol = Chem.MolFromPDBFile(ligandFile, removeHs=False)
        
        # # Add positions to mol before trying to use it for fingerprints
        
        # if mol is not None:
        #     # Generate Morgan fingerprint
        #     fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        #     print(f"Ligand: {ligandFile}, Fingerprint bits set: {fp.GetNumOnBits()}") # Optional: print fingerprint info
            
        # else:
        #     print(f"Warning: Could not load ligand {ligandFile} with RDKit.")
        #     continue # Skip this ligand if RDKit fails
        
        
        structure_ligand = parser.get_structure("ligand", ligandFile)    
        model_ligand = structure_ligand[0]
        
        # bpdb_file =bio_pdb.PDBFile.read(ligandFile)
        # tc5b = bpdb_file.get_structure()

        for chain in model_ligand:
            # Create a new chain ID for the ligand if needed
            # Find a chain ID that's not already in use
            existing_chain_ids = [c.id for c in model]
            ligand_chain_id = 'Z'  # Default
            for c in 'ZXVULKJIHGFEDCBA':
                if c not in existing_chain_ids:
                    ligand_chain_id = c
                    break

            new_chain = Chain.Chain(ligand_chain_id)
            newCount =0 
            for residue in chain:
                # Skip unwanted HETATM residues
                if residue.resname in UNWANTED_HETATMS:
                    continue
                new_residue = residue.copy()
                new_chain.add(new_residue)
                newCount += 1
                
            if newCount==0:
                print(f" ", end='')
                continue
            newModel = CensorProtein(model,identicalChains, new_chain, boxDistance_nm=2.5)
            if newModel is None:
                print(f"No residues within distance of ligand center for {pdbID}")
                continue
            #save the new model to a file
            io = PDBIO()
            io.set_structure(newModel)
            baseLigandFile = os.path.basename(ligandFile)
            io.save(os.path.join(outputFolder, f"{baseLigandFile}__{ligand_chain_id}.pdb"))
            cc+=1
        
    return structure, model 

pdbFolder =r'D:\PythonProj\pdbFiles\BindingMOAD_2020_processed\BindingMOAD_2020_processed\pdb_protein'
ligandFolder =r'D:\PythonProj\pdbFiles\BindingMOAD_2020_processed\BindingMOAD_2020_processed\pdb_superligand'
outputFolder =r'D:\PythonProj\pdbFiles\censored'


# Search for PDB IDs with ligands
folder =r'D:\PythonProj\pdbFiles\BindingMOAD_2020_processed\BindingMOAD_2020_processed\pdb_protein'

files=glob.glob(os.path.join(folder, '*.pdb'))
pdb_ids = []
#format = 1o6z_1_protein.pdb
for file in files:
    # Extract the PDB ID from the filename
    pdb_id = os.path.basename(file)
    if pdb_id[0]!='.' :
        pdb_id = pdb_id.split('_')[0]
    if len(pdb_id) == 4:
        pdb_ids.append(pdb_id.upper())

for cc,pdb_id in enumerate(pdb_ids):
    #try:
        load_MOAD_pdb(pdb_id)
        print('.',end='')
        if (cc+1)%100==0:
            print(cc)
    # except KeyboardInterrupt as e:
    #     raise(e)
    # except Exception as e:
    #     print('x',end='')
    #     pass
 



.......................... ...... .   .  .............. ........ . .   ......... ..  .  ....  .  . . .  ......  ......    .   .........99
.................    .  .   ............    . ........................  ..  .  .... . ...    ........................ . . .......199
..... . . .... .... .............    .....  ........ .  .................................... .. . ....... ............299
....... . ...  .  .  ... .................... ...... ....  ...........    ...........................   ................399
..........  .  .......    ... . . . ...   ....... ..   .  .. ............ . .  ....    ..  .........  .  ........ . . .    ... .. . ..............499
..    .... ................ . ... ..  .  ......    . . ...  ....  .. . . . . . .....  .  . . ........ .....  .......  . .....  .............   ..599
  .  ........   ...  .  .  .  .  .  . . .  .  . . . . . . .  .  .  .  .  .  . .  .  ..  .  ..    ....    ..... .. ....  .  . .  . . . . .  ...............................  ....  .699
.

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PythonProj\\pdbFiles\\BindingMOAD_2020_processed\\BindingMOAD_2020_processed\\pdb_superligand\\1D01_1_superlig_0.pdb'

In [ ]:


load_MOAD_pdb('1A05')

No residues in ligand 1A05


(<Structure id=protein>, <Model id=0>)

In [13]:
pdb_id

'1A0Q'

In [ ]:


if False:
    structure, model, ligands_info = loadPDF(pdb_path)
    # Display the model
    displayModel(model)


    for info in ligands_info:
        grid3D_red, grid3D_green, grid3D_blue, mask, voxelSize, minX, minY, minZ   = CreateStack(model, info, boxSize_nm=2, gridSize=128)
        visualization, mask_proj = create_depth_visualization(grid3D_red, grid3D_green, grid3D_blue,mask, num_sections=5)
        
        file_name = f"__visualization.png"

        # Convert numpy array to PIL Image and save
        img = Image.fromarray((visualization * 255).astype(np.uint8))
        img.save(file_name)
        print( np.max(np.max(visualization,axis=0),axis=0)  , np.min(np.min(visualization,axis=0),axis=0) )
        print( np.max( grid3D_red), np.max(grid3D_green), np.max(grid3D_blue) )
        plt.figure(figsize=(2, 2))
        plt.imshow(visualization)
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.show()

        print(f"Saved visualization to {file_name}")